In [1]:
import os
import sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import pandas as pd
from pycona import *
from pycona.benchmarks import *

### Comparing interactive CA systems

For the introductory example, we only used GrowAcq algorithm, with its default settings. In this notebook, we will first run and compare different interactive CA algorithms, and then we will see how we can customize their configuration.

In [2]:
# we use the running example on nurse rostering from the introductory tutorial
instance, oracle = construct_nurse_rostering(3, 2, 8, 2)
print(instance)

ProblemInstance: 

Name nurse_rostering_nurses8_shifts3_days2_nurses_per_shift2.

Parameters {'shifts_per_day': 3, 'num_days': 2, 'num_nurses': 8, 'nurses_per_shift': 2}.

Variables: [[[shifts[0,0,0] shifts[0,0,1]]
  [shifts[0,1,0] shifts[0,1,1]]
  [shifts[0,2,0] shifts[0,2,1]]]

 [[shifts[1,0,0] shifts[1,0,1]]
  [shifts[1,1,0] shifts[1,1,1]]
  [shifts[1,2,0] shifts[1,2,1]]]].

Language: [(AV4) == (AV5), (AV4) != (AV5), (AV4) < (AV5), (AV4) > (AV5), (AV4) >= (AV5), (AV4) <= (AV5)].


#### Customizing Interactive CA systems 

Each interactive CA algorithm can be configured in different ways for each of the 3 main subcomponents of interactive CA systems:

- qgen: An instance of QGenBase, default is None. The query generation system to be used to generate top-level queries.
- find_scope: An instance of FindScopeBase, default is None. The FindScope system to be used for finding the scope of violated constraints.
- findc: An instance of FindCBase, default is None. The FindConstraint system to be used for finding the exact violated constraints in the given scopes.

For each of the subcomponents several choices are implemented, based on the literature. Each subcomponent of CA is implemented in a class, with different implementations of that subcomponent subclassing it.

Until now, we used the default settings in **PyConA** for interactive CA systems. We will now focus on how to customize the behaviour of a given Interactive CA system. 

#### Using a custom CA Environment

In [3]:
# Creating probabilistic environment, using the default settings for all subcomponents
env = ProbaActiveCAEnv(qgen=PQGen(), find_scope=FindScope2(), findc=FindC())

ga2 = GrowAcq(env)
learned_instance = ga2.learn(instance, oracle=oracle, verbose=1)

Running growacq with <pycona.active_algorithms.mquacq2.MQuAcq2 object at 0x000001D6D4478820> as inner algorithm

Learned 0 constraints in 0 queries.
...L..
Learned 1 constraints in 5 queries.
....L....L.
Learned 3 constraints in 14 queries.
....L.....L.....L..
Learned 6 constraints in 30 queries.
.......L...L....L...L
Learned 10 constraints in 47 queries.
.....L....L.....L....L..L
Learned 15 constraints in 67 queries.
........L....L..
Learned 17 constraints in 81 queries.
.......L....L.....L..
Learned 20 constraints in 99 queries.
....L....L....
Learned 22 constraints in 111 queries.
.......L........L...L....
Learned 25 constraints in 133 queries.
.......L....L..L..L...
Learned 29 constraints in 151 queries.
.......L.....L...L...L......L....
Learned 34 constraints in 179 queries.


In [4]:
# Lets use the older 'FindScope' instead of FindScope2, and see what the effect is
env = ProbaActiveCAEnv(qgen=PQGen(), find_scope=FindScope(), findc=FindC())

ga1 = GrowAcq(env)
learned_instance = ga1.learn(instance, oracle=oracle, verbose=1)

Running growacq with <pycona.active_algorithms.mquacq2.MQuAcq2 object at 0x000001D6D464FE80> as inner algorithm

Learned 0 constraints in 0 queries.
.....L..
Learned 1 constraints in 7 queries.
......L.......L.
Learned 3 constraints in 21 queries.
......L........L.........L..
Learned 6 constraints in 46 queries.
.........L.....L....L....L
Learned 10 constraints in 68 queries.
........L.......L........L......L...L
Learned 15 constraints in 100 queries.
...........L.........L..
Learned 17 constraints in 122 queries.
.........L...........L.........L..
Learned 20 constraints in 153 queries.
...........L.............L.....
Learned 22 constraints in 182 queries.
................L..........L.........L..
Learned 25 constraints in 219 queries.
.............L........L......L.......L...
Learned 29 constraints in 256 queries.
............L..........L......L.............L.....L.....
Learned 34 constraints in 307 queries.


In [5]:
# we can compare the detailed statistics:
pd.concat([ga2.env.metrics.short_statistics, 
           ga1.env.metrics.short_statistics], keys=["growacq findscope2", "growacq findscope"])

,,CL,tot_q,top_lvl_q,tfs_q,tfc_q,avg_q_size,avg_gen_time,avg_t,max_t,tot_t,conv
growacq findscope2,0,34,179,68,77,34,4.6257,0.1687,0.0865,0.6577,15.4857,1
growacq findscope,0,34,307,69,206,32,3.9121,0.1354,0.0417,0.5501,12.7876,1


#### Comparing different algorithms

First the classic QuAcq, MQuAcq and MQuAcq2 algorithms.

_(by default they do not use a probabilistic environment, though you can change that)_

In [6]:
# some patience needed
qa = QuAcq()
learned_instance = qa.learn(instance, oracle=oracle)

mqa = MQuAcq()
learned_instance = mqa.learn(instance, oracle=oracle)

mqa2 = MQuAcq2()
learned_instance = mqa2.learn(instance, oracle=oracle)

Then, the more recent 'GrowAcq' meta-algorithm

(includes a probabilistic environment that guides query generation)

In [7]:
ga24 = GrowAcq()  # AAAI24 version, decision tree with features
learned_instance = ga24.learn(instance, oracle=oracle)

ga23 = GrowAcq(ProbaActiveCAEnv(classifier=CountsPredictor()))  # CP23 version, counting based probabilities
learned_instance = ga23.learn(instance, oracle=oracle)

In [8]:
# we can compare the detailed statistics:
out = pd.concat([a.env.metrics.short_statistics for a in [qa,mqa,mqa2,ga23,ga24]])
out.index = ["qa","mqa","mqa2","ga23","ga24"]
out

,CL,tot_q,top_lvl_q,tfs_q,tfc_q,avg_q_size,avg_gen_time,avg_t,max_t,tot_t,conv
qa,34,244,38,173,33,6.2992,1.0003,0.1682,1.8389,41.0412,1
mqa,34,157,65,24,68,2.7389,0.8791,0.0633,1.7251,9.9337,1
mqa2,34,201,37,125,32,4.6468,0.7745,0.0685,1.6736,13.7633,1
ga23,34,90,88,0,2,5.6667,0.1079,0.1145,0.4404,10.3072,1
ga24,34,83,81,0,2,5.3976,0.1343,0.1447,0.7583,12.0081,1
